In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("train.csv")
train_x = data.drop('Survived', axis = 1)
print(train_x)
train_x['Age'] = train_x['Age'].fillna(train_x['Age'].median())
train_x = train_x.drop(['PassengerId', 'Name', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis = 1)
d = {'male': 0 ,'female': 1}
train_x['Sex'] = train_x['Sex'].apply(lambda x:d[x])
train_y = data['Survived']
theta = np.zeros((1, 5))
train_x = train_x.values

train_y = train_y.values.reshape(891, 1)

     PassengerId  Pclass                                               Name  \
0              1       3                            Braund, Mr. Owen Harris   
1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2              3       3                             Heikkinen, Miss. Laina   
3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4              5       3                           Allen, Mr. William Henry   
5              6       3                                   Moran, Mr. James   
6              7       1                            McCarthy, Mr. Timothy J   
7              8       3                     Palsson, Master. Gosta Leonard   
8              9       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9             10       2                Nasser, Mrs. Nicholas (Adele Achem)   
10            11       3                    Sandstrom, Miss. Marguerite Rut   
11            12       1                           B

In [6]:
test_x = pd.read_csv("test.csv")
test_x = test_x.drop(['PassengerId', 'Name', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis = 1)
test_x['Sex'] = test_x['Sex'].apply(lambda x:d[x])
test_x['Age'] = test_x['Age'].fillna(test_x['Age'].median())
test_x = test_x.values
test_y = pd.read_csv("gender_submission.csv")
test_y = test_y.drop(['PassengerId'], axis = 1)
test_y = test_y.values

In [7]:
ones = np.ones((891, 1))
train_x = np.concatenate((ones, train_x), axis=1)
print(train_x.shape)
ones = np.ones((418, 1))
test_x = np.concatenate((ones, test_x), axis = 1)
print(test_x.shape)

(891, 5)
(418, 5)


In [8]:
def hypothesis(x, theta):
    h = np.zeros([891, 1])
    for i in range(len(x)):
        a = x[i]
        h[i][0] = 1 / (1 + np.exp(-np.sum(np.dot(a, theta.T))) - 1)
    return h

def hypothesis_test(x, theta):
    h = np.zeros([418, 1])
    for i in range(len(x)):
        a = x[i]
        h[i][0] = 1 / (1 + np.exp(-np.sum(np.dot(a, theta.T))) - 1)
    return h

def costfn(x, y, theta):
    predictions = hypothesis(x, theta)
    c = (-y) * math.log(predictions) - (1 - y) * math.log(1 - predictions)
    return c

def gradient(x, y, theta, alpha, iterations):
    for i in range(iterations):
        h = hypothesis(x, theta)
        loss = h - y
        g = loss.T.dot(x) / len(y)
        theta = theta - alpha * g
    return theta

def decide(prob):
    for i in range(len(prob)):
        if prob[i] < 0.5:
            prob[i] = 0
        else:
            prob[i] = 1
    return prob

In [9]:
theta = gradient(train_x, train_y, theta, 0.005, 7000)
prob = hypothesis_test(test_x, theta)
z = prob.mean()
for i in range(len(prob)):
    if prob[i] == 0:
        prob[i] = z
prob = decide(hypothesis_test(test_x, theta))

In [10]:
score = 0
for i in range(len(prob)):
    if prob[i] == test_y[i]:
        score = score + 1
print(score)

387


In [11]:
accuracy = 387 / 418 * 100
print(accuracy)

92.58373205741627


In [16]:
#F1 score
tp = 0
tn = 0
fn = 0
fp = 0
for i in range(len(prob)):
    if prob[i] == 1 and test_y[i] == 1:
        tp = tp + 1
    elif prob[i] == 0 and test_y[i] == 0:
        tn = tn + 1
    elif prob[i] == 1 and test_y[i] == 0:
        fp = fp + 1
    elif prob[i] == 0 and test_y[i] == 1:
        fn = fn + 1

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision * recall) / (precision + recall)
print(f1)

0.8872727272727273
